In [1]:
from ase.io import read
from ase.io.lammpsdata import write_lammps_data
from ase.calculators.lammps import Prism, convert

from lammps import lammps
import os
import sys
import numpy as np
sys.path.append("/home/dux/")
sys.path.append("/home/dux/surface_sampling/sgmc_surf")

from htvs.djangochem.pgmols.utils import surfaces

os.environ["LAMMPS_COMMAND"] = "/home/dux/lammps/src/lmp_serial"
os.environ["LAMMPS_POTENTIALS"] = "/home/dux/lammps/potentials/"

main_dir = "/home/dux/surface_sampling/sgmc_surf/GaN"

In [2]:
FLIP_ORDER = [(1, 0, 0), (2, 0, 0), (2, 1, 1)]

class PrismLocal:
    """The representation of the unit cell in LAMMPS

    The main purpose of the prism-object is to create suitable
    string representations of prism limits and atom positions
    within the prism.

    :param cell: cell in ase coordinate system
    :param pbc: periodic boundaries
    :param tolerance: precision for skewness test

    **Implementation**

    LAMMPS requires triangular matrixes without a strong tilt.
    Therefore the 'Prism'-object contains three coordinate systems:

    - ase_cell (the simulated system in the ASE coordination system)
    - lammps_tilt (ase-cell rotated to be an lower triangular matrix)
    - lammps_cell (same volume as tilted cell, but reduce edge length)

    The translation between 'ase_cell' and 'lammps_cell' is down with a
    rotation matrix 'rot_mat' obtained from a QR decomposition.  The
    transformation between 'lammps_tilt' and 'lammps_cell' is done by
    changing the off-diagonal elements.  'flip' saves the modified
    elements for later reversal.  All vectors except positions are just
    rotated with 'rot_mat'.  Positions are rotated and wrapped into
    'lammps_cell'.  Translating results back from LAMMPS needs first the
    unwrapping of positions.  Then all vectors and the unit-cell are
    rotated back into the ASE coordination system.  This can fail as
    depending on the simulation run LAMMPS might have changed the
    simulation box significantly.  This is for example a problem with
    hexagonal cells.  LAMMPS might also wrap atoms across periodic
    boundaries, which can lead to problems for example NEB
    calculations.
    """

    # !TODO: derive tolerance from cell-dimensions
    def __init__(self, cell, pbc=(True, True, True), tolerance=1.0e-8):
        # Use QR decomposition to get the lammps cell
        #    rot_mat * lammps_tilt^T = ase_cell^T
        # => lammps_tilt * rot_mat^T = ase_cell
        # => lammps_tilt             = ase_cell * rot_mat
        qtrans, ltrans = np.linalg.qr(cell.T, mode="complete")
        self.rot_mat = qtrans
        self.lammps_tilt = ltrans.T
        self.ase_cell = cell
        self.tolerance = tolerance
        self.pbc = pbc

        # LAMMPS requires positive values on the diagonal of the
        # triangluar matrix -> mirror if necessary
        for i in range(3):
            if self.lammps_tilt[i][i] < 0.0:
                mirror_mat = np.eye(3)
                mirror_mat[i][i] = -1.0
                self.lammps_tilt = np.dot(mirror_mat, self.lammps_tilt.T).T
                self.rot_mat = np.dot(self.rot_mat, mirror_mat)

        print(f"lammps_tilt {self.lammps_tilt}")
        print(f"rot_mat {self.rot_mat}")

        self.lammps_cell = self.lammps_tilt.copy()
        self.flip = np.array(
            [
                abs(self.lammps_cell[i][j] / self.lammps_tilt[k][k]) > 0.5
                and self.pbc[k]
                for i, j, k in FLIP_ORDER
            ]
        )

        print(f"flip {self.flip}")

        for iteri, (i, j, k) in enumerate(FLIP_ORDER):
            if self.flip[iteri]:
                change = self.lammps_cell[k][k]
                change *= np.sign(self.lammps_cell[i][j])
                self.lammps_cell[i][j] -= change
        
        print(f"final lammps cell {self.lammps_cell}")

In [3]:
# define necessary file locations
lammps_in_file = f"{main_dir}/lammps.in"
potential_file = f"GaN.tersoff"
atoms = ["Ga", "N"]
lammps_out_file = f"{main_dir}/lammps.out"
cif_from_lammps_file = f"{main_dir}/lammps_from_cif.cif"

supercell_atoms = read('GaN_hexagonal.cif')*(3,3,3)
supercell_atoms.write('GaN_hexagonal_3x3.cif')

bulk_slab, surface_atoms = surfaces.surface_from_bulk(supercell_atoms, [0,0,0,-1], size=[3,3], vacuum=10)

print(f"bulk positions before", bulk_slab.get_positions())

# set surface atoms from the other side
all_atoms = np.arange(len(bulk_slab))
curr_surf_atoms = bulk_slab.get_surface_atoms()
new_surf_atoms = np.setdiff1d(all_atoms, curr_surf_atoms)
bulk_slab.set_surface_atoms(new_surf_atoms)
# invert the positions
bulk_slab.set_scaled_positions(1 - bulk_slab.get_scaled_positions())
bulk_slab.write("/home/dux/surface_sampling/sgmc_surf/GaN/GaN_0001_with_scaled_postions.cif")
print(f"bulk positions after", bulk_slab.get_positions())

lammps_data_file = f"{main_dir}/lammps_from_cell.data"
bulk_slab.write(
    lammps_data_file, format="lammps-data", units="metal", atom_style="atomic"
)

cell = bulk_slab.get_cell()
print(cell)
p = Prism(cell)
units = 'metal'
convert(p.get_lammps_prism(), "distance", "ASE", units)

p = PrismLocal(cell)

correct_cell = cell
# print("lammps prism", lammps_tilt[(0, 1, 2, 1, 2, 2), (0, 1, 2, 0, 0, 1)])

# write_lammps_data(sys.stdout, bulk_slab, specorder=None, force_skew=False,
#                       prismobj=None, velocities=False, units="metal",
#                       atom_style='atomic')

/home/dux/miniconda3/envs/surface_sampling/lib/python3.8/site-packages/catkit/gen/surface.py:334: UserWarning: Your slab has no bulk atoms and may be too thin to identify surface atoms correctly. This may cause surface adsorption site identification to fail.
  warnings.warn(


bulk positions before [[ 0.          0.         10.        ]
 [ 1.60814504  0.92846297 10.64577292]
 [ 1.60814504  0.92846297 12.619981  ]
 [ 0.          0.         13.26575392]
 [ 1.60814503  2.78538891 10.        ]
 [ 3.21629007  3.71385188 10.64577292]
 [ 3.21629007  3.71385188 12.619981  ]
 [ 1.60814503  2.78538891 13.26575392]
 [ 3.21629007  5.57077781 10.        ]
 [ 4.8244351   6.49924078 10.64577292]
 [ 4.8244351   6.49924078 12.619981  ]
 [ 3.21629007  5.57077781 13.26575392]
 [ 3.21629007  0.         10.        ]
 [ 4.8244351   0.92846297 10.64577292]
 [ 4.8244351   0.92846297 12.619981  ]
 [ 3.21629007  0.         13.26575392]
 [ 4.8244351   2.78538891 10.        ]
 [ 6.43258014  3.71385188 10.64577292]
 [ 6.43258014  3.71385188 12.619981  ]
 [ 4.8244351   2.78538891 13.26575392]
 [ 6.43258014  5.57077781 10.        ]
 [ 8.04072517  6.49924078 10.64577292]
 [ 8.04072517  6.49924078 12.619981  ]
 [ 6.43258014  5.57077781 13.26575392]
 [ 6.43258014  0.         10.        ]
 [ 

In [6]:
lammps_data_file = f"{main_dir}/lammps_from_cif.data"
# cif_slab = read("/home/dux/surface_sampling/sgmc_surf/GaN/GaN_0001_with_scaled_postions.cif")
cif_slab = read("/home/dux/surface_sampling/sgmc_surf/GaN/GaN_0001_3x3_pymatgen/runs500_temp1_adsatoms12_alpha0.995_20220518-172644/optim_slab_run_500.cif")



cif_slab.cell = correct_cell
cell = cif_slab.get_cell()
print(cell)
p = Prism(cell)
units = 'metal'
convert(p.get_lammps_prism(), "distance", "ASE", units)

p = PrismLocal(cell)

cif_from_cif_file = f"{main_dir}/cif_from_cif.cif"
cif_slab.write(cif_from_cif_file)
print(f"cif scaled positions", cif_slab.get_positions())

cif_slab.write(
    lammps_data_file, format="lammps-data", units="metal", atom_style="atomic"
)
write_lammps_data(sys.stdout, cif_slab, specorder=None, force_skew=False,
                      prismobj=None, velocities=False, units="metal",
                      atom_style='atomic')

Cell([[9.648870209999998, 0.0, 0.0], [4.824435104999999, 8.35616671967889, 0.0], [0.0, 0.0, 23.26575391688]])
lammps_tilt [[ 9.64887021  0.          0.        ]
 [ 4.8244351   8.35616672  0.        ]
 [ 0.          0.         23.26575392]]
rot_mat [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
flip [False False False]
final lammps cell [[ 9.64887021  0.          0.        ]
 [ 4.8244351   8.35616672  0.        ]
 [ 0.          0.         23.26575392]]
cif scaled positions [[ 0.          0.         13.26569384]
 [12.86517608  7.42771287 12.62006789]
 [12.86517608  7.42771287 10.64573211]
 [ 0.          0.         10.00010616]
 [ 3.21630608  5.57080555 13.26569384]
 [11.25704716  4.64235188 12.62006789]
 [11.25704716  4.64235188 10.64573211]
 [ 3.21630608  5.57080555 10.00010616]
 [ 1.60812892  2.78536099 13.26569384]
 [ 9.64887     1.85690733 12.62006789]
 [ 9.64887     1.85690733 10.64573211]
 [ 1.60812892  2.78536099 10.00010616]
 [ 6.43261216  0.         13.26569384]
 [ 9.64891824  7.42771287 

In [8]:
# write current surface into lammps.data
slab = cif_slab
slab.write(
    lammps_data_file, format="lammps-data", units="metal", atom_style="atomic"
)
TEMPLATE = """
clear
atom_style atomic
units metal
boundary p p p
atom_modify sort 0 0.0

# read_data /path/to/data.data
read_data {}

### set bulk
group bulk id <= 36
group surface id > 36

### interactions
pair_style tersoff
# pair_coeff * * /path/to/potential Atom1 Atom2
pair_coeff * * {} {} {}
mass 1 69.723000
mass 2 14.007000

### run
reset_timestep 0
velocity surface create 100.0 87287 dist gaussian
fix 1 all nvt temp 20.0 0.1 $(100.0*dt)

# set bulk forces and velocities to 0
fix 2 bulk setforce 0.0 0.0 0.0
velocity bulk set 0.0 0.0 0.0

thermo 10 # output thermodynamic variables every N timesteps

thermo_style custom step temp press ke pe xy xz yz
thermo_modify flush yes format float %23.16g
# min_style cg
# minimize 1e-5 1e-5 500 10000

timestep 0.001
run 3000

# write_data /path/to/data.out
write_data {}
print "_end of MD_"
log /dev/stdout

"""

# write lammps.in file
with open(lammps_in_file, "w") as f:
    f.writelines(
        TEMPLATE.format(lammps_data_file, potential_file, *atoms, lammps_out_file)
    )

lmp = lammps()
print("LAMMPS Version: ", lmp.version())

# run the LAMMPS here
lmp.file(lammps_in_file)
lmp.close()

# Read from LAMMPS out
opt_slab = read(lammps_out_file, format="lammps-data", style="atomic")

atomic_numbers_dict = {1: 31, 2: 7}  # 1: Ga, 2: N
actual_atomic_numbers = [
    atomic_numbers_dict[x] for x in opt_slab.get_atomic_numbers()
]
print(f"actual atomic numbers {actual_atomic_numbers}")
# correct_lammps = new_slab.copy()
opt_slab.set_atomic_numbers(actual_atomic_numbers)
opt_slab.calc = slab.calc
opt_slab.write(cif_from_lammps_file)

LAMMPS (29 Sep 2021)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
LAMMPS Version:  20210929
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0.0000000 0.0000000 0.0000000) to (9.6488702 8.3561667 23.265754) with tilt (4.8244351 0.0000000 0.0000000)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  48 atoms
  read_data CPU = 0.000 seconds
36 atoms in group bulk
12 atoms in group surface
Reading tersoff potential file GaN.tersoff with DATE: 2007-10-22
Neighbor list info ...
  update every 1 steps, delay 10 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 5.1
  ghost atom cutoff = 5.1
  binsize = 2.55, bins = 6 4 10
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair tersoff, perpetual
      attributes: full, newton on
      pair build: 